In [1]:
import cv from "@techstark/opencv-js";
import { Tensor, InferenceSession } from "onnxruntime-node";
import fs from "fs";
import { detectImage } from "./code/detect";
const { Canvas, createCanvas, Image, ImageData, loadImage } = require('canvas') as typeof import('canvas');
const { JSDOM } = require('jsdom') as typeof import('jsdom');


// Configs
const modelName = "./models/lpr-8n.onnx";
const modelInputShape = [1, 3, 640, 640];
const topk = 100;
const iouThreshold = 0.45;
const scoreThreshold = 0.25;

cv.onRuntimeInitialized = async () => {
    const fileBuffer =  fs.readFileSync(modelName);
    const u8:Uint8Array = new Uint8Array(fileBuffer);
    const options: InferenceSession.SessionOptions = {
      logSeverityLevel: 3,
      logVerbosityLevel: 3,
    }
 
    const yolov8 = await InferenceSession.create(u8, options); //{executionProviders: ['cpu']}

    const inputNames = yolov8.inputNames;
    const outputNames = yolov8.outputNames;
    console.log("inputNames: ", inputNames);
    console.log("outputNames: ", outputNames);
    
   // console.log("yolov8: ", session);

    //warmup
    const tensor = new Tensor(
      "float32",
      new Float32Array(modelInputShape.reduce((a, b) => a * b)),
      modelInputShape
    );
    const mmcSession = await yolov8.run({images: tensor});
   
    const session = { net: yolov8, nms: yolov8};


    //session['net']  = await yolov8.run({ images: tensor });
   /// session.net.run
/*
    const feeds: Record<string,any> = {};
    feeds[session.inputNames[0]] = tensor;
const session2 = await session.run(feeds);

const session3 = session2[session.outputNames[0]];
//console.log("session1: ", session2[session.outputNames[0]]);
      */
      
       try {
        installDOM();
        const image = await loadImage('./sample-images/car-with-licenseplate.jpeg')
      
        const canvas = createCanvas(640, 640);
        const ctx = canvas.getContext('2d');
        ctx.drawImage(image, 0, 0, 640, 640);
        
        await detectImage(
            image,
            canvas,
            session,
            topk,
            iouThreshold,
            scoreThreshold,
            modelInputShape
          );
          
          fs.writeFileSync('./sample-images/output2.jpg', canvas.toBuffer('image/jpeg'));
          console.log('done');
      //  const preprocessResult = preprocessing(image, 640, 640);
       // console.log("preprocessResult: ", preprocessResult);
      } catch (error) {
        console.log("error: ", error);
      }

}

function installDOM() {
    const dom = new JSDOM();
    global.document = dom.window.document;
    // The rest enables DOM image and canvas and is provided by node-canvas
    global.Image = Image;
    global.HTMLCanvasElement = Canvas;
    global.ImageData = ImageData;
    global.HTMLImageElement = Image;
}




inputNames:  [ 'images' ]
outputNames:  [ 'output0' ]
xRatio 1
yRatio 1.3333333333333333
before run
after run
output0: h {
  dims: [ 1, 5, 8400 ],
  type: 'float32',
  data: Float32Array(42000) [
     5.914968013763428,    7.1198410987854, 23.545604705810547,
    42.809146881103516, 43.564876556396484,  43.97624588012695,
     45.02202606201172,  46.34746170043945,  50.90314483642578,
     85.07113647460938,   96.7187271118164, 102.42646789550781,
    107.81242370605469, 112.66954040527344, 119.71017456054688,
    123.70437622070312, 137.88943481445312, 148.91250610351562,
    152.51451110839844, 164.81883239746094, 167.14450073242188,
    166.23428344726562, 168.03866577148438, 172.82028198242188,
     178.7190704345703,  188.9889373779297, 202.61178588867188,
     208.4152374267578,  214.1649627685547, 228.37510681152344,
    244.64248657226562, 254.92819213867188, 258.34527587890625,
     263.9659729003906,      267.900390625,  279.2474060058594,
    288.13653564453125, 295.55242919

UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/dist/converter.js:111:19)
    at Object.scheduleInvalidateResolutionsOfFailedLookupLocations (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:122719:55)
    at scheduleInvalidateResolutionOfFailedLookupLocation (/Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121553:22)
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:121617:11
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5810:11
    at /Users/patrick/.nvm/versions/node/v18.15.0/lib/node_modules/tslab/node_modules/@tslab/typescript-for-tslab/lib/typescript.js:5560:101
    a

: 